In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

the_blind_flight_synapse_drive_ps_1_path = kagglehub.competition_download('the-blind-flight-synapse-drive-ps-1')
haragr10_blindflightunet_pytorch_default_1_path = kagglehub.model_download('haragr10/blindflightunet/PyTorch/default/1')
haragr10_blindflightfinal_pytorch_2_1_path = kagglehub.model_download('haragr10/blindflightfinal/PyTorch/2/1')

print('Data source import complete.')


# Sample Pipeline:
  - Load small training set (20 maps) from:

        train/images/*.png

        train/labels/*.json     # contains true class grid

        train/velocities/*.json # boost; we ignore it in this baseline

  - Train a tiny NN that predicts a 20x20 class grid from the image.

        * It treats all terrains together (no terrain-specific modeling).

        * It only uses per-cell average colour (super crude).

  - For each test image:

        * Predict class grid.

        * Find start (class 3) and goal (class 4).

        * Compute a path with a dumb cost model (ignoring boosts).

        * Convert path to an 'lrud' sequence and write submission_baseline.csv
        

In [ ]:
!pip install albumentations --no-deps
!pip install "numpy<2"
!pip install -U segmentation-models-pytorch

In [ ]:
import os, random, numpy as np, cv2, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image, ImageDraw
import segmentation_models_pytorch as smp
from pathlib import Path
from tqdm import tqdm

# ---------------- CONFIG ----------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
GRID_SIZE = 20
TILE_SIZE = 40
IMG_SIZE = 800
BATCH_SIZE = 16
ACCUM_STEPS = 4

# ---------------- ASSETS ----------------
INPUT_ROOT = Path("/kaggle/input")
ASSET_ROOT = next((p for p in INPUT_ROOT.rglob("assets") if p.is_dir()), Path("assets"))

def load_asset(path):
    try:
        return Image.open(path).convert("RGBA").resize((TILE_SIZE, TILE_SIZE))
    except:
        return Image.new("RGBA", (TILE_SIZE, TILE_SIZE), (128,128,128,255))

ASSETS = {
    0: {"floor":[load_asset(ASSET_ROOT/"lab/t2_floor.png")],
        "wall":[load_asset(ASSET_ROOT/"lab/t2_wall.png")],
        "hazard":[load_asset(ASSET_ROOT/"lab/t2_glue.png")],
        "start":load_asset(ASSET_ROOT/"lab/t2_drone.png"),
        "goal":load_asset(ASSET_ROOT/"lab/t2_goal.png")},
    1: {"floor":[load_asset(ASSET_ROOT/"forest/t0_dirt.png")],
        "wall":[load_asset(ASSET_ROOT/"forest/t0_tree.png")],
        "hazard":[load_asset(ASSET_ROOT/"forest/t0_puddle.png")],
        "start":load_asset(ASSET_ROOT/"forest/t0_startship.png"),
        "goal":load_asset(ASSET_ROOT/"forest/t0_goal.png")},
    2: {"floor":[load_asset(ASSET_ROOT/"desert/t1_sand.png")],
        "wall":[load_asset(ASSET_ROOT/"desert/t1_rocks.png")],
        "hazard":[load_asset(ASSET_ROOT/"desert/t1_quicksand.png")],
        "start":load_asset(ASSET_ROOT/"desert/t1_rover.png"),
        "goal":load_asset(ASSET_ROOT/"desert/t1_goal.png")}
}

# ---------------- DATASET ----------------
class StableSkewDataset(Dataset):
    def __init__(self, length=10000):
        self.length = length
        self.color = A.Compose([
            A.HueSaturationValue(5, 10, 5, p=0.4),
            A.RGBShift(10, 10, 10, p=0.3),
            A.RandomBrightnessContrast(0.1, 0.1, p=0.4),
        ])
        self.blur = A.OneOf([
            A.GaussianBlur((3, 5), p=1.0),
            A.MotionBlur((3, 7), p=1.0)
        ], p=0.4)
        self.final = A.Compose([
            A.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ToTensorV2()
        ])

    def __len__(self): return self.length

    def mild_skew(self, img, mask):
        if random.random() > 0.4: return img, mask
        h, w = img.shape[:2]
        limit = 18
        pts1 = np.float32([[0, 0], [w, 0], [0, h], [w, h]])
        pts2 = np.float32([
            [random.randint(0, limit), random.randint(0, limit)],
            [w - random.randint(0, limit), random.randint(0, limit)],
            [random.randint(0, limit), h - random.randint(0, limit)],
            [w - random.randint(0, limit), h - random.randint(0, limit)]
        ])
        M = cv2.getPerspectiveTransform(pts1, pts2)
        img = cv2.warpPerspective(img, M, (w, h), borderMode=cv2.BORDER_REFLECT_101)
        mask = cv2.warpPerspective(mask, M, (w, h), flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_REFLECT_101)
        return img, mask

    def __getitem__(self, idx):
        t_id = random.choice([0, 1, 2])
        assets = ASSETS[t_id]
        grid = np.random.choice([0, 1, 2], (GRID_SIZE, GRID_SIZE), p=[0.7, 0.2, 0.1])
        s, g = random.sample([(r, c) for r in range(GRID_SIZE) for c in range(GRID_SIZE)], 2)
        grid[s] = 3; grid[g] = 4

        # Grid Rotation
        k = random.choice([0, 1, 2, 3])
        if k > 0: grid = np.rot90(grid, k=k)

        canvas = Image.new("RGBA", (IMG_SIZE, IMG_SIZE), (0, 0, 0, 255))
        draw = ImageDraw.Draw(canvas)

        for r in range(GRID_SIZE):
            for c in range(GRID_SIZE):
                x, y = c * TILE_SIZE, r * TILE_SIZE
                canvas.paste(random.choice(assets["floor"]), (x, y))
                cell = grid[r, c]
                if cell == 1: canvas.paste(random.choice(assets["wall"]), (x, y))
                elif cell == 2: canvas.paste(random.choice(assets["hazard"]), (x, y))
                elif cell == 3: canvas.paste(assets["start"], (x, y))
                elif cell == 4: canvas.paste(assets["goal"], (x, y))

                # Black Border
                draw.rectangle([x, y, x + TILE_SIZE - 1, y + TILE_SIZE - 1], outline=(0, 0, 0), width=2)

        img = np.array(canvas.convert("RGB"))
        mask_full = cv2.resize(grid.astype(np.uint8), (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
        img, mask_full = self.mild_skew(img, mask_full)

        img = self.color(image=img)["image"]
        img = self.blur(image=img)["image"]
        img = self.final(image=img)["image"]

        mask = cv2.resize(mask_full, (GRID_SIZE, GRID_SIZE), interpolation=cv2.INTER_NEAREST)
        return img, torch.from_numpy(mask).long(), torch.tensor(t_id).long()

# ---------------- MODEL ----------------
class MultiHeadUNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.unet = smp.Unet(encoder_name="resnet34", encoder_weights="imagenet", in_channels=3, classes=5)
        c = self.unet.encoder.out_channels[-1]
        self.terrain = nn.Sequential(
            nn.AdaptiveAvgPool2d(1), nn.Flatten(),
            nn.Linear(c,64), nn.ReLU(), nn.Linear(64,3)
        )

    def forward(self,x):
        feats = self.unet.encoder(x)
        terr = self.terrain(feats[-1])
        dec = self.unet.decoder(feats)
        seg = self.unet.segmentation_head(dec)
        seg = torch.nn.functional.interpolate(seg,(GRID_SIZE,GRID_SIZE), mode="bilinear",align_corners=False)
        return seg, terr

# ---------------- COUNT LOSS ----------------
class CountConsistencyLoss(nn.Module):
    def forward(self, logits):
        probs = torch.softmax(logits,1)
        return sum(((probs[:,c].sum((1,2))-1)**2).mean() for c in [3,4])

# ---------------- TRAIN ----------------
def train():
    dl = DataLoader(StableSkewDataset(), BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    model = MultiHeadUNet().to(DEVICE)
    if torch.cuda.device_count() > 1: model = nn.DataParallel(model)

    opt = optim.AdamW(model.parameters(), 1e-3, weight_decay=1e-2)
    scaler = GradScaler()
    sched = optim.lr_scheduler.ReduceLROnPlateau(opt, "min", 0.5, 2)

    # FIX: Use floats (1.0, 5.0) to prevent Scalar Type Error
    ce = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 5.0, 5.0, 50.0, 50.0], device=DEVICE))
    terr_ce = nn.CrossEntropyLoss()
    count_loss = CountConsistencyLoss()

    print("🚀 Starting Training... (Press SQUARE STOP BUTTON to save & exit early)")

    try:
        for ep in range(15):
            model.train()
            loop = tqdm(dl, desc=f"Epoch {ep+1}/15")
            epoch_loss = 0

            for i, (x, y, t) in enumerate(loop):
                x, y, t = x.to(DEVICE), y.to(DEVICE), t.to(DEVICE)

                with autocast():
                    seg, terr = model(x)

                    l_seg = ce(seg, y)
                    l_terr = 0.5 * terr_ce(terr, t)

                    l_count = torch.tensor(0.0, device=DEVICE)
                    if ep >= 5:
                        l_count = 0.2 * count_loss(seg)

                    loss = (l_seg + l_terr + l_count) / ACCUM_STEPS

                scaler.scale(loss).backward()

                if (i+1) % ACCUM_STEPS == 0:
                    scaler.step(opt); scaler.update(); opt.zero_grad()
                    epoch_loss += loss.item() * ACCUM_STEPS

                    # Update Progress Bar with ALL losses
                    loop.set_postfix({
                        "Seg": f"{l_seg.item():.3f}",
                        "Terr": f"{l_terr.item():.3f}",
                        "Cnt": f"{l_count.item():.3f}",
                        "Tot": f"{loss.item()*ACCUM_STEPS:.3f}"
                    })

            sched.step(epoch_loss/len(dl))

    except KeyboardInterrupt:
        print("\n🛑 Training Interrupted by User! Saving current state...")

    # Save logic runs even if interrupted
    torch.save(model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict(),
               "unet_stable_skew_final.pth")
    print("✅ Model Saved: unet_stable_skew_final.pth")

if __name__=="__main__":
    train()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import random
import albumentations as A
from PIL import Image, ImageDraw, ImageEnhance
from pathlib import Path
from torch.utils.data import Dataset

# --- CONFIG ---
GRID_SIZE = 20
TILE_SIZE = 40
IMG_SIZE = 800
INPUT_ROOT = Path("/kaggle/input")
TEST_IMG_DIR = Path("/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/test/images")

# --- 1. ASSETS ---
ASSET_ROOT = None
for path in INPUT_ROOT.rglob("assets"):
    if path.is_dir():
        ASSET_ROOT = path
        break
if not ASSET_ROOT: ASSET_ROOT = Path("assets")

def load_asset(path):
    try:
        return Image.open(path).convert("RGBA").resize((TILE_SIZE, TILE_SIZE))
    except FileNotFoundError:
        return Image.new("RGBA", (TILE_SIZE, TILE_SIZE), (128, 128, 128, 255))

ASSETS = {
    0: {"floor": [load_asset(ASSET_ROOT/"lab/t2_floor.png")], "wall": [load_asset(ASSET_ROOT/"lab/t2_wall.png")], "hazard": [load_asset(ASSET_ROOT/"lab/t2_glue.png")], "start": load_asset(ASSET_ROOT/"lab/t2_drone.png"), "goal": load_asset(ASSET_ROOT/"lab/t2_goal.png")},
    1: {"floor": [load_asset(ASSET_ROOT/"forest/t0_dirt.png")], "wall": [load_asset(ASSET_ROOT/"forest/t0_tree.png")], "hazard": [load_asset(ASSET_ROOT/"forest/t0_puddle.png")], "start": load_asset(ASSET_ROOT/"forest/t0_startship.png"), "goal": load_asset(ASSET_ROOT/"forest/t0_goal.png")},
    2: {"floor": [load_asset(ASSET_ROOT/"desert/t1_sand.png")], "wall": [load_asset(ASSET_ROOT/"desert/t1_rocks.png")], "hazard": [load_asset(ASSET_ROOT/"desert/t1_quicksand.png")], "start": load_asset(ASSET_ROOT/"desert/t1_rover.png"), "goal": load_asset(ASSET_ROOT/"desert/t1_goal.png")}
}

# --- 2. REFINED DATASET CLASS ---
class RefinedSkewDataset(Dataset):
    def __init__(self, length=20):
        self.length = length

        # A. Global Transforms (Rotation + Tint)
        self.global_transform = A.Compose([
            A.RandomRotate90(p=0.5), # Added explicit rotation augmentation
            A.HueSaturationValue(hue_shift_limit=5, sat_shift_limit=10, val_shift_limit=5, p=0.4),
            A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=0.3),
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.4),
        ])

        # B. Blur (High Probability)
        self.blur_transform = A.OneOf([
            A.GaussianBlur(blur_limit=(3, 7), p=0.8),
            A.MotionBlur(blur_limit=(3, 9), p=0.8),
        ], p=0.8) # Increased to 80% to ensure it appears often

    def apply_tint_patches(self, image_np):
        """
        Creates localized circles (radius ~3 tiles) with SUBTLE color shifts.
        """
        h_img, w_img, _ = image_np.shape
        mask = np.zeros((h_img, w_img), dtype=np.uint8)

        num_patches = random.randint(0, 3)
        if num_patches == 0: return image_np

        for _ in range(num_patches):
            radius = random.randint(100, 140)
            cx, cy = random.randint(0, w_img), random.randint(0, h_img)
            cv2.circle(mask, (cx, cy), radius, 255, -1)

        img_hsv = cv2.cvtColor(image_np, cv2.COLOR_RGB2HSV).astype(np.int32)

        # --- REDUCED INTENSITY HERE ---
        # Hue +/- 10 (was 20), Sat +20 (was 40)
        h_shift = random.randint(-10, 10)
        s_shift = random.randint(0, 20)
        v_shift = random.randint(-10, 10)

        img_hsv[:, :, 0] = (img_hsv[:, :, 0] + h_shift) % 180
        img_hsv[:, :, 1] = np.clip(img_hsv[:, :, 1] + s_shift, 0, 255)
        img_hsv[:, :, 2] = np.clip(img_hsv[:, :, 2] + v_shift, 0, 255)

        img_tinted = cv2.cvtColor(img_hsv.astype(np.uint8), cv2.COLOR_HSV2RGB)

        mask_bool = mask > 0
        image_np[mask_bool] = img_tinted[mask_bool]

        return image_np

    def __len__(self): return self.length

    def __getitem__(self, idx):
        # 1. Base Grid Generation
        t_id = random.choice([0, 1, 2])
        assets = ASSETS[t_id]
        grid_label = np.random.choice([0, 1, 2], size=(GRID_SIZE, GRID_SIZE), p=[0.7, 0.2, 0.1])
        p1, p2 = random.sample([(r, c) for r in range(GRID_SIZE) for c in range(GRID_SIZE)], 2)
        grid_label[p1] = 3; grid_label[p2] = 4

        # Grid Rotation (Structural)
        grid_label = np.rot90(grid_label, k=random.choice([0, 1, 2, 3]))

        full_w, full_h = GRID_SIZE * TILE_SIZE, GRID_SIZE * TILE_SIZE
        canvas = Image.new("RGBA", (full_w, full_h), (0, 0, 0, 255))
        draw = ImageDraw.Draw(canvas)

        for r in range(GRID_SIZE):
            for c in range(GRID_SIZE):
                x, y = c * TILE_SIZE, r * TILE_SIZE
                cell = grid_label[r, c]
                canvas.paste(random.choice(assets["floor"]), (x, y))
                if cell == 1: canvas.paste(random.choice(assets["wall"]), (x, y))
                elif cell == 2: canvas.paste(random.choice(assets["hazard"]) if assets.get("hazard") else assets["wall"][0], (x, y))
                elif cell == 3: canvas.paste(assets["start"], (x, y))
                elif cell == 4: canvas.paste(assets["goal"], (x, y))

                # DRAW BLACK BORDER
                draw.rectangle([x, y, x + TILE_SIZE - 1, y + TILE_SIZE - 1], outline=(0, 0, 0, 255), width=2)

        image_np = np.array(canvas.convert("RGB"))

        # 2. Apply Local Color Patches
        image_np = self.apply_tint_patches(image_np)

        # 3. Geometric Skew (Increased Severity + Frequency)
        if random.random() < 0.8: # Fixed probability (was mismatch in comments)
            pts1 = np.float32([[0,0], [full_w,0], [0,full_h], [full_w,full_h]])

            # INCREASED Skew Limit to 55 (was 40)
            limit = 55
            tl = [random.randint(0, limit), random.randint(0, limit)]
            tr = [full_w - random.randint(0, limit), random.randint(0, limit)]
            bl = [random.randint(0, limit), full_h - random.randint(0, limit)]
            br = [full_w - random.randint(0, limit), full_h - random.randint(0, limit)]

            pts2 = np.float32([tl, tr, bl, br])
            M = cv2.getPerspectiveTransform(pts1, pts2)

            image_np = cv2.warpPerspective(image_np, M, (full_w, full_h), borderValue=(0,0,0))

            # Apply Global Augmentations (Blur, Rotate, etc.)
            image_np = self.global_transform(image=image_np)['image']
            image_np = self.blur_transform(image=image_np)['image']

        return image_np

# --- 3. DISPLAY FUNCTION ---
def show_training_data_preview():
    train_dataset = RefinedSkewDataset(length=20)

    try:
        test_files = list(TEST_IMG_DIR.glob("*.png"))
        if len(test_files) < 10: test_files = []
    except Exception:
        test_files = []

    rows = 10
    fig, axes = plt.subplots(rows, 2, figsize=(14, 4 * rows))
    plt.subplots_adjust(wspace=0.1, hspace=0.2)

    print("🎨 Previewing Data Generation...")
    print("LEFT: Synthetic Training Data (Subtle Patch + Strong Skew + Blur)")
    print("RIGHT: Real Test Data (If available)")

    for i in range(rows):
        img_train = train_dataset[i]
        axes[i, 0].imshow(img_train)
        axes[i, 0].set_title(f"TRAIN: Skew+Blur+Patch", fontsize=10, color='green')
        axes[i, 0].axis('off')

        if test_files:
            img_path = random.choice(test_files)
            img_test = Image.open(img_path).convert("RGB")
            axes[i, 1].imshow(img_test)
            axes[i, 1].set_title(f"TEST: {img_path.name}", fontsize=10, color='blue')
            axes[i, 1].axis('off')
        else:
            axes[i, 1].axis('off')
            axes[i, 1].text(0.5, 0.5, "No Test Data Found", ha='center')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    show_training_data_preview()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import torch
import cv2
import json
import heapq
import random
import albumentations as A
import torch.nn as nn
import segmentation_models_pytorch as smp
from albumentations.pytorch import ToTensorV2
from pathlib import Path
from PIL import Image

# --- CONFIGURATION ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Ensure this matches the name of the file you saved
MODEL_PATH = "unet_stable_skew_final.pth"
TEST_IMG_DIR = Path("/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/test/images")
TEST_VEL_DIR = Path("/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/test/velocities")

# --- PHYSICS CONSTANTS ---
TERRAIN_CONFIG = {
    0: {
        "name": "LAB",
        "costs": {0: 1.0, 1: float('inf'), 2: 3.0, 3: 1.0, 4: 2.0}
    },
    1: {
        "name": "FOREST",
        "costs": {0: 1.5, 1: float('inf'), 2: 2.8, 3: 1.5, 4: 2.5}
    },
    2: {
        "name": "DESERT",
        "costs": {0: 1.2, 1: float('inf'), 2: 3.7, 3: 1.2, 4: 2.2}
    }
}

# --- MODEL DEFINITION (Must Match Training Script) ---
class MultiHeadUNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Matches the training parameters
        self.unet = smp.Unet(encoder_name="resnet34", encoder_weights=None,
                             in_channels=3, classes=5)

        # Matches the 'self.terrain' head from training
        c = self.unet.encoder.out_channels[-1]
        self.terrain = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(c, 64),
            nn.ReLU(),
            nn.Linear(64, 3)
        )

    def forward(self, x):
        features = self.unet.encoder(x)
        # Use the same head name as training
        terrain_logits = self.terrain(features[-1])

        decoder_out = self.unet.decoder(features)
        seg_logits = self.unet.segmentation_head(decoder_out)

        # Interpolate to 20x20 during inference for planning
        seg_logits = torch.nn.functional.interpolate(
            seg_logits, size=(20, 20),
            mode='bilinear', align_corners=False
        )
        return seg_logits, terrain_logits

# --- PATH PLANNER ---
class StarkPathPlanner:
    def __init__(self, grid_classes, boost_matrix, terrain_id):
        self.grid = grid_classes
        self.boost = boost_matrix
        self.costs = TERRAIN_CONFIG[terrain_id]["costs"]
        self.rows, self.cols = grid_classes.shape

    def heuristic(self, a, b):
        # Manhattan distance
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    def solve(self, start, end):
        pq = [(0, start)]
        g_scores = {start: 0.0}
        came_from = {}

        while pq:
            _, current = heapq.heappop(pq)
            if current == end:
                return self.reconstruct_path(came_from, current)

            r, c = current
            # Neighbors: Up, Down, Left, Right
            neighbors = [((r-1, c), 'u'), ((r+1, c), 'd'),
                         ((r, c-1), 'l'), ((r, c+1), 'r')]

            for next_cell, move_dir in neighbors:
                nr, nc = next_cell

                # Bounds check
                if not (0 <= nr < self.rows and 0 <= nc < self.cols): continue

                # Wall check (Class 1 is Wall)
                if self.grid[nr, nc] == 1: continue

                # Cost Calculation: Base Cost - Boost
                cell_type = self.grid[nr, nc]
                base = self.costs.get(cell_type, 1.0)

                # Boost logic: subtract boost from cost, minimum cost 0.01
                boost_val = self.boost[nr, nc]
                cost = max(base - boost_val, 0.01)

                new_g = g_scores[current] + cost

                if new_g < g_scores.get(next_cell, float('inf')):
                    g_scores[next_cell] = new_g
                    came_from[next_cell] = (current, move_dir)
                    priority = new_g + self.heuristic(next_cell, end)
                    heapq.heappush(pq, (priority, next_cell))

        return None, []

    def reconstruct_path(self, came_from, current):
        path_str = []
        path_coords = [current]
        while current in came_from:
            prev, move = came_from[current]
            path_str.append(move)
            path_coords.append(prev)
            current = prev
        return "".join(path_str[::-1]), path_coords[::-1]

# --- VISUALIZER ---
def visualize_check(num_samples=3):
    # Load Model
    model = MultiHeadUNet().to(DEVICE)
    try:
        model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
        print("✅ Model loaded successfully.")
    except Exception as e:
        print(f"❌ Failed to load model: {e}")
        return

    model.eval()

    transform = A.Compose([
        A.Resize(800, 800),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])

    # Grab Test Files
    try:
        files = list(TEST_IMG_DIR.glob("*.png"))
        if not files:
            print("Warning: No test images found. Using dummy data?")
            return
        samples = random.sample(files, min(len(files), num_samples))
    except Exception as e:
        print(f"Error accessing test directory: {e}")
        return

    for img_path in samples:
        # 1. Inference
        original_image = np.array(Image.open(img_path).convert("RGB"))
        t_img = transform(image=original_image)['image'].unsqueeze(0).to(DEVICE)

        with torch.no_grad():
            seg, terr = model(t_img)
            # Seg is already 20x20 due to model definition above
            grid = torch.argmax(seg, dim=1).squeeze().cpu().numpy()
            t_id = torch.argmax(terr, dim=1).item()

        # 2. Load Velocity (if available)
        boost = np.zeros((20, 20))
        json_path = TEST_VEL_DIR / f"{img_path.stem}.json"
        if json_path.exists():
            try:
                with open(json_path) as f:
                    boost = np.array(json.load(f)['boost'])
            except:
                pass # Default to zero boost if error

        # 3. Plan Path
        # Find Start (3) and Goal (4)
        starts = np.argwhere(grid == 3)
        goals = np.argwhere(grid == 4)

        path_str = "NO PATH"
        path_coords = []

        if len(starts) > 0 and len(goals) > 0:
            # Take the first found start/goal if duplicates exist
            start_node = tuple(starts[0])
            goal_node = tuple(goals[0])

            planner = StarkPathPlanner(grid, boost, t_id)
            path_str, path_coords = planner.solve(start_node, goal_node)
            if not path_str: path_str = "NO PATH"

        # 4. Plotting
        fig, ax = plt.subplots(1, 2, figsize=(14, 6))

        # Left: Image + Path
        ax[0].imshow(original_image)
        t_name = TERRAIN_CONFIG.get(t_id, {}).get("name", "UNKNOWN")
        ax[0].set_title(f"Terrain: {t_name} | Path: {path_str[:20]}...")

        if path_coords:
            # Scale 20x20 coordinates to 800x800 image space for plotting
            scale_y = original_image.shape[0] / 20
            scale_x = original_image.shape[1] / 20

            # Extract Y and X coordinates
            py, px = zip(*path_coords)

            # Shift by 0.5 to center on tile and scale up
            plot_x = [(x + 0.5) * scale_x for x in px]
            plot_y = [(y + 0.5) * scale_y for y in py]

            ax[0].plot(plot_x, plot_y, 'r-', linewidth=4, label="Path", alpha=0.8)
            ax[0].scatter(plot_x[0], plot_y[0], c='lime', s=100, label='Start', zorder=5)
            ax[0].scatter(plot_x[-1], plot_y[-1], c='magenta', s=100, label='Goal', zorder=5)
            ax[0].legend()
        else:
            ax[0].text(400, 400, "NO PATH FOUND", color='red', fontsize=20,
                       ha='center', va='center', backgroundcolor='black')

        # Right: Predicted Grid
        # Custom color map for clarity: Floor, Wall, Haz, Start, Goal
        cmap = plt.get_cmap('tab10', 5)
        im = ax[1].imshow(grid, cmap=cmap, vmin=-0.5, vmax=4.5)
        ax[1].set_title("Predicted Segmentation Grid (20x20)")

        # Add legend for grid
        cbar = plt.colorbar(im, ax=ax[1], ticks=[0, 1, 2, 3, 4])
        cbar.ax.set_yticklabels(['Floor', 'Wall', 'Hazard', 'Start', 'Goal'])

        plt.tight_layout()
        plt.show()

# RUN IT
if __name__ == "__main__":
    visualize_check(30)

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp
import json
import heapq
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

# --- 1. CONFIGURATION ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Ensure this matches your saved file name
MODEL_PATH = "unet_stable_skew_final.pth"
TEST_IMG_DIR = Path("/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/test/images")
TEST_VEL_DIR = Path("/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/test/velocities")
BATCH_SIZE = 64  # High throughput
NUM_WORKERS = 4

# --- 2. PHYSICS CONSTANTS ---
TERRAIN_CONFIG = {
    0: {"name": "LAB", "costs": {0: 1.0, 1: float('inf'), 2: 3.0, 3: 1.0, 4: 2.0}},
    1: {"name": "FOREST", "costs": {0: 1.5, 1: float('inf'), 2: 2.8, 3: 1.5, 4: 2.5}},
    2: {"name": "DESERT", "costs": {0: 1.2, 1: float('inf'), 2: 3.7, 3: 1.2, 4: 2.2}}
}

# --- 3. MODEL DEFINITION (Must Match Training) ---
class MultiHeadUNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Matches the training parameters
        self.unet = smp.Unet(encoder_name="resnet34", encoder_weights=None,
                             in_channels=3, classes=5)

        # Matches the 'self.terrain' head from training
        c = self.unet.encoder.out_channels[-1]
        self.terrain = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(c, 64),
            nn.ReLU(),
            nn.Linear(64, 3)
        )

    def forward(self, x):
        features = self.unet.encoder(x)
        # Use the same head name as training
        terrain_logits = self.terrain(features[-1])

        decoder_out = self.unet.decoder(features)
        seg_logits = self.unet.segmentation_head(decoder_out)

        # Interpolate to 20x20 during inference for planning
        seg_logits = torch.nn.functional.interpolate(
            seg_logits, size=(20, 20),
            mode='bilinear', align_corners=False
        )
        return seg_logits, terrain_logits

# --- 4. DATASET ---
class TestDataset(Dataset):
    def __init__(self, img_dir, vel_dir):
        self.img_paths = sorted(list(img_dir.glob("*.png")))
        self.vel_dir = vel_dir
        self.transform = A.Compose([
            A.Resize(800, 800),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ])

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image_id = img_path.stem

        image = np.array(Image.open(img_path).convert("RGB"))
        t_img = self.transform(image=image)['image']

        v_path = self.vel_dir / f"{image_id}.json"
        if v_path.exists():
            with open(v_path, 'r') as f:
                boost = np.array(json.load(f)['boost'], dtype=np.float32)
        else:
            boost = np.zeros((20, 20), dtype=np.float32)

        return t_img, boost, image_id

# --- 5. OPTIMIZED DIJKSTRA PLANNER ---
class StarkPathPlanner:
    def __init__(self, grid_classes, boost_matrix, terrain_id):
        self.grid = grid_classes
        self.boost = boost_matrix
        self.costs = TERRAIN_CONFIG[terrain_id]["costs"]
        self.rows, self.cols = grid_classes.shape

    def solve(self, start, end):
        # Priority Queue: (Cost, Current_Cell)
        # Note: No Heuristic here. Just pure cost accumulation.
        pq = [(0, start)]

        # Track minimum cost to reach each cell
        min_costs = {start: 0.0}
        came_from = {}

        while pq:
            current_cost, current = heapq.heappop(pq)

            # Optimization: If we found a path to 'current' that is cheaper already, skip
            if current_cost > min_costs.get(current, float('inf')):
                continue

            if current == end:
                return self.reconstruct_path(came_from, current)

            r, c = current
            neighbors = [((r-1, c), 'u'), ((r+1, c), 'd'), ((r, c-1), 'l'), ((r, c+1), 'r')]

            for next_cell, move_dir in neighbors:
                nr, nc = next_cell

                # Bounds check
                if not (0 <= nr < self.rows and 0 <= nc < self.cols):
                    continue

                # Wall check
                if self.grid[nr, nc] == 1:
                    continue

                # PHYSICS ENGINE: Cost = Base - Boost
                base = self.costs.get(self.grid[nr, nc], 1.0)

                # Floor cost at 0.01 to prevent infinite loops or negative weights
                # Dijkstra requires non-negative edge weights!
                step_cost = max(base - self.boost[nr, nc], 0.01)

                new_cost = current_cost + step_cost

                # If this path is cheaper than any previous path to this neighbor
                if new_cost < min_costs.get(next_cell, float('inf')):
                    min_costs[next_cell] = new_cost
                    came_from[next_cell] = (current, move_dir)
                    heapq.heappush(pq, (new_cost, next_cell))

        return None

    def reconstruct_path(self, came_from, current):
        path = []
        while current in came_from:
            prev, move = came_from[current]
            path.append(move)
            current = prev
        return "".join(path[::-1])

# --- 6. MAIN EXECUTION ---
def generate_submission():
    print(f"⏳ Loading Model from {MODEL_PATH}...")

    # Load Weights (CPU safe first)
    if not Path(MODEL_PATH).exists():
        print(f"❌ Error: Model file '{MODEL_PATH}' not found!")
        return

    checkpoint = torch.load(MODEL_PATH, map_location="cpu")
    model = MultiHeadUNet()

    # Fix DataParallel keys if present (e.g. 'module.unet...')
    if list(checkpoint.keys())[0].startswith('module.'):
        checkpoint = {k[7:]: v for k, v in checkpoint.items()}

    try:
        model.load_state_dict(checkpoint)
        print("✅ Weights loaded successfully.")
    except RuntimeError as e:
        print(f"❌ Weight mismatch! Check class definition.\n{e}")
        return

    if torch.cuda.device_count() > 1:
        print(f"🔥 Dual-GPU Activated: {torch.cuda.device_count()} GPUs")
        model = nn.DataParallel(model)

    model.to(DEVICE)
    model.eval()

    # Dataset & Loader
    # Check if directories exist
    if not TEST_IMG_DIR.exists():
        print("❌ Test Image Directory not found. Check path.")
        return

    test_ds = TestDataset(TEST_IMG_DIR, TEST_VEL_DIR)
    loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

    results = []
    print(f"🚀 Starting Dijkstra Inference on {len(test_ds)} images...")

    with torch.no_grad():
        for batch_imgs, batch_boosts, batch_ids in tqdm(loader, desc="Planning"):
            batch_imgs = batch_imgs.to(DEVICE)

            # 1. Model Inference
            seg_logits, terrain_logits = model(batch_imgs)

            # 2. Resize is already done inside forward() -> (20x20)

            # 3. CPU Transfer
            grids = torch.argmax(seg_logits, dim=1).cpu().numpy()
            t_ids = torch.argmax(terrain_logits, dim=1).cpu().numpy()
            boosts = batch_boosts.numpy()

            # 4. Dijkstra Loop (CPU)
            for i in range(len(batch_ids)):
                image_id = batch_ids[i]
                grid = grids[i]
                t_id = t_ids[i].item()
                boost = boosts[i]

                path_str = ""
                starts = np.argwhere(grid == 3)
                goals = np.argwhere(grid == 4)

                if len(starts) > 0 and len(goals) > 0:
                    planner = StarkPathPlanner(grid, boost, t_id)
                    # Solves for global optimum now
                    res = planner.solve(tuple(starts[0]), tuple(goals[0]))
                    if res: path_str = res

                results.append({"image_id": image_id, "path": path_str})

    # Save
    df = pd.DataFrame(results)
    df.to_csv("submission.csv", index=False)
    print(f"✅ Submission saved with {len(df)} rows.")

if __name__ == "__main__":
    generate_submission()

In [ ]:

# Load the dataframe
df = pd.read_csv('/kaggle/working/submission.csv')

# Inspect the dataframe

# Fill missing values with 'd'
df_filled = df.fillna('d')

# Verify the changes
print(df_filled.info())
print(df_filled[df['path'].isna()].head()) # Check rows that were originally missing

# Save the modified dataframe to a new CSV
df_filled.to_csv('submission_filled.csv', index=False)